In [3]:
import pandas as pd
from glob import glob
import os

df = pd.read_csv("Orthogroups.csv", sep=";")
df =df[["Orthogroup","Rhimi59"]]
df

#Crear una lista para almacenar las nuevas filas
nuevas_filas = []
#iterar sobre cada una de las filas del df
for index, row in df.iterrows():
    ortho = row["Orthogroup"]
    ids   = str(row["Rhimi59"]).split(",") # separar los IDs por coma
    
    #iterar sobre cada ID y agregarlo como una nueva fila junto con el ortogrupo
    for id in ids:
        nuevas_filas.append([ortho, id.strip()]) #eliminar los espacios en blanco alrededor de los IDs si los hay
#crear un nuevo df con las nuevas filas
nuevo_df = pd.DataFrame(nuevas_filas, columns=["Orthogroup", "Rhimi59"])
nuevo_df

Antes de hacer el merge como hay genes que pueden tener más de un vecino (son muy pocos pero luego pueden fastidiar por eso se deben procesar los archivos teniendo en cuenta que puede haber genes con más de un vecino)

In [4]:
# Obtener la lista de todos los archivos CSV en el directorio actual
archivos_csv = glob("*vecinos.csv")

# Iterar sobre cada archivo CSV
for archivo in archivos_csv:
    # Leer el archivo CSV
    df = pd.read_csv(archivo)
    
    # Crear una lista para almacenar las nuevas filas
    nuevas_filas = []
    
    # Lista para almacenar genes con más de un vecino
    genes_con_mas_de_un_vecino = []
    
    # Iterar sobre cada fila del DataFrame
    for index, row in df.iterrows():
        gen_id = row["Gen_ID"]
        vecinos = row["Vecinos"]
        
        # Verificar si hay más de dos valores separados por coma en la columna de Vecinos
        if "," in str(vecinos) and len(vecinos.split(",")) > 1:
            # Dividir los valores separados por coma y crear una nueva fila para cada uno
            for vecino in vecinos.split(","):
                nuevas_filas.append([gen_id, vecino.strip()])
            # Agregar el gen a la lista de genes con más de un vecino
            genes_con_mas_de_un_vecino.append(gen_id)
        else:
            # Si no hay más de dos valores, simplemente agregar la fila existente
            nuevas_filas.append([gen_id, vecinos])
    
    # Crear un nuevo DataFrame con las nuevas filas
    nuevo_df = pd.DataFrame(nuevas_filas, columns=["Gen_ID", "Vecinos"])
    
    # Guardar el nuevo DataFrame en un nuevo archivo CSV
    nuevo_archivo = f"{archivo.split('.')[0]}_actualizado.csv"
    nuevo_df.to_csv(nuevo_archivo, index=False)

    # Imprimir los genes con más de un vecino
    if genes_con_mas_de_un_vecino:
        print(f"En el archivo {archivo}, los siguientes genes tienen más de un vecino:")
        for gen in genes_con_mas_de_un_vecino:
            print(gen)

    print(f"Archivo actualizado guardado como: {nuevo_archivo}")


Archivo actualizado guardado como: Aspnid1_filtered_vecinos_actualizado.csv
En el archivo Cunech1_vecinos.csv, los siguientes genes tienen más de un vecino:
183389
Archivo actualizado guardado como: Cunech1_vecinos_actualizado.csv
En el archivo Hesve2finisherSC_vecinos.csv, los siguientes genes tienen más de un vecino:
1405097
1331872
1336460
1225215
1407221
1321639
1080470
83607
Archivo actualizado guardado como: Hesve2finisherSC_vecinos_actualizado.csv
En el archivo Liccor1_vecinos.csv, los siguientes genes tienen más de un vecino:
3901
9409
9410
8935
7360
7795
7006
6128
12527
12607
4829
8515
4933
7439
8630
3777
2811
10797
12142
2424
6992
6730
2374
4087
7367
5492
505
9527
300
8657
7038
6255
5357
2733
8456
6344
13300
6445
7917
2884
2885
5215
6131
4558
1747
3515
2881
2882
2883
4660
11925
11226
9872
4921
7303
3071
11874
3417
8142
7390
8712
9725
1011
5592
8780
11859
2618
6967
4807
9021
9186
9411
9412
96
8888
2718
9366
13060
43
4348
7429
1558
4417
12424
3524
6357
6125
5949
3735
3972
5138


El archivo de Phycomyces por cualquier razón los ids no aparecen como enteros. Hay que transformarlos

In [5]:
# Leer el archivo CSV de phycomyces
df = pd.read_csv("Phybl2_vecinos_actualizado.csv")

# Eliminar las filas duplicadas
df = df.drop_duplicates()

# Convertir los identificadores a enteros
df["Gen_ID"] = df["Gen_ID"].astype(int)
df["Vecinos"] = df["Vecinos"].astype(int)

# Guardar el DataFrame modificado en un nuevo archivo CSV
df.to_csv("Phybl2_vecinos_actualizado.csv", index=False)


**OBTENER LOS ORTOGRUPOS A LOS QUE PERTENECE CADA VECINO**. Obtener un csv para cada organismo.

In [6]:
# Cargar el archivo CSV
df = pd.read_csv("Orthogroups.csv", sep=";")
# Reemplazar los valores NaN con cero en todas las columnas
df = df.fillna(0)

# Iterar sobre cada columna que representa un organismo, excluyendo la primera columna "Orthogroup"
for col in df.columns[1:]:
    # Crear un DataFrame para el organismo actual
    organismo_df = df[["Orthogroup", col]]
    
    # Crear una lista para almacenar las nuevas filas
    nuevas_filas = []
    
    # Iterar sobre cada fila del DataFrame del organismo actual
    for index, row in organismo_df.iterrows():
        ortho = row["Orthogroup"]
        ids   = str(row[col]).split(",")  # Separar los IDs por coma
        
        # Iterar sobre cada ID y agregarlo como una nueva fila junto con el ortogrupo
        for id in ids:
           # nuevas_filas.append([ortho, int(id.strip())])  # Eliminar los espacios en blanco alrededor de los IDs si los hay
            nuevas_filas.append([ortho, int(id)])
    # Crear un nuevo DataFrame con las nuevas filas para el organismo actual
    nuevo_df = pd.DataFrame(nuevas_filas, columns=["Orthogroup", col])
    
    # Guardar el nuevo DataFrame en un nuevo archivo CSV con un nombre que incluya el nombre del organismo
    nuevo_df.to_csv(f"{col}_ortho.csv", index=False)
nuevo_df

,Orthogroup,Spoumb1
0,OG0000000,439232
1,OG0000000,507207
2,OG0000000,519485
3,OG0000000,521449
4,OG0000000,546167
...,...,...
4038,OG0003859,543668
4039,OG0003860,491621
4040,OG0003860,557429
4041,OG0003861,516110


MERGE DE LOS VECINOS Y DE LOS ORTOGRUPOS

este es el bueno

In [7]:
# Obtener la lista de todos los archivos CSV en el directorio actual
archivos_ortho = glob("*_ortho.csv")
archivos_vecinos = glob("*_vecinos_actualizado.csv")

# Iterar sobre cada archivo ortho
for archivo_ortho in archivos_ortho:
    # Obtener el primer campo del nombre del archivo ortho
    nombre_campo_ortho = os.path.basename(archivo_ortho).split('_')[0]
    
    # Buscar el archivo de vecinos que tenga el mismo primer campo en el nombre
    archivo_vecinos = next((f for f in archivos_vecinos if os.path.basename(f).startswith(nombre_campo_ortho)), None)
    
    # Verificar si se encontró el archivo de vecinos correspondiente
    if archivo_vecinos:
        # Leer los archivos CSV
        ortho = pd.read_csv(archivo_ortho, sep=",")
        vecinos = pd.read_csv(archivo_vecinos, sep=",")

        # Realizar el merge
        merge_resultado = pd.merge(vecinos, ortho, left_on="Gen_ID", right_on=nombre_campo_ortho, how="left")
        merge_resultado = pd.merge(merge_resultado, ortho, left_on="Vecinos", right_on=nombre_campo_ortho, how="left")

        # Filtrar las filas donde al menos una de las columnas Orthogroup esté vacía
        merge_filtrado = merge_resultado[(merge_resultado["Orthogroup_x"].notnull()) & (merge_resultado["Orthogroup_y"].notnull())]

        # Guardar el resultado en un nuevo archivo CSV
        nombre_salida = f"{nombre_campo_ortho}_merge.csv"  # Generar un nombre de archivo de salida
        merge_filtrado.to_csv(nombre_salida, index=False)
    else:
        print(f"No se encontró un archivo de vecinos correspondiente para {archivo_ortho}. Saltando este archivo.")


Quiero extraer las columnas Orthogroup_x y Orthogroup_y de cada archivo *_merge.csv y luego concatenar los resultados de las columnas para tener sólo dos columnas con todos los valores. Y después quiero seleccionar las combinaciones únicas (es decir parejas únicas de Orthogroup_x y Orthogroup_y)

In [9]:
# Obtener la lista de todos los archivos *_merge.csv en el directorio actual
archivos_merge = glob("*_merge.csv")

# Lista para almacenar los DataFrames de las columnas Orthogroup_x y Orthogroup_y
columnas_orthogroup = []

# Leer y procesar cada archivo *_merge.csv
for archivo_merge in archivos_merge:
    # Leer el archivo CSV
    df = pd.read_csv(archivo_merge)
    
    # Extraer las columnas Orthogroup_x y Orthogroup_y
    columnas_orthogroup.append(df[['Orthogroup_x', 'Orthogroup_y']])

# Concatenar los DataFrames de las columnas Orthogroup_x y Orthogroup_y
df_concatenado = pd.concat(columnas_orthogroup)

# Seleccionar las combinaciones únicas de Orthogroup_x y Orthogroup_y
combinaciones_unicas = df_concatenado.drop_duplicates()

# Mostrar las combinaciones únicas
print(combinaciones_unicas)


     Orthogroup_x Orthogroup_y
0       OG0001460    OG0003027
1       OG0001133    OG0000034
2       OG0001868    OG0001566
3       OG0001461    OG0000394
4       OG0001462    OG0000038
...           ...          ...
1218    OG0000951    OG0000148
1221    OG0003852    OG0001918
1223    OG0001772    OG0000533
1225    OG0003816    OG0000076
1228    OG0003182    OG0000014

[9965 rows x 2 columns]


In [37]:
df_concatenado

,Orthogroup_x,Orthogroup_y
0,OG0001460,OG0003027
1,OG0001133,OG0000034
2,OG0001868,OG0001566
3,OG0001461,OG0000394
4,OG0001462,OG0000038
...,...,...
1241,OG0001222,OG0000276
1242,OG0000214,OG0000210
1243,OG0002295,OG0001218
1244,OG0000038,OG0000191


In [10]:
import glob

# Obtener la lista de todos los archivos *_merge.csv en el directorio actual
archivos_merge = glob.glob("*_merge.csv")

# Lista para almacenar los DataFrames de las columnas Orthogroup_x y Orthogroup_y
columnas_orthogroup = []

# Leer y procesar cada archivo *_merge.csv
for archivo_merge in archivos_merge:
    # Leer el archivo CSV
    df = pd.read_csv(archivo_merge)
    
    # Extraer las columnas Orthogroup_x y Orthogroup_y
    columnas_orthogroup.append(df[['Orthogroup_x', 'Orthogroup_y']])

# Concatenar los DataFrames de las columnas Orthogroup_x y Orthogroup_y
df_concatenado = pd.concat(columnas_orthogroup)

# Crear una nueva columna que contenga tuplas de Orthogroup_x y Orthogroup_y
df_concatenado['combinacion'] = df_concatenado.apply(lambda row: tuple(sorted([row['Orthogroup_x'], row['Orthogroup_y']])), axis=1)

# Eliminar las duplicaciones basadas en la columna 'combinacion'
combinaciones_unicas = df_concatenado.drop_duplicates(subset='combinacion')

# Eliminar la columna 'combinacion' ya que ya no es necesaria
combinaciones_unicas = combinaciones_unicas.drop(columns=['combinacion'])
combinaciones_unicas.to_csv("combinaciones_unicas.csv")
# Mostrar las combinaciones únicas
print(combinaciones_unicas)


     Orthogroup_x Orthogroup_y
0       OG0001460    OG0003027
1       OG0001133    OG0000034
2       OG0001868    OG0001566
3       OG0001461    OG0000394
4       OG0001462    OG0000038
...           ...          ...
983     OG0000012    OG0000561
1020    OG0002283    OG0000920
1067    OG0002600    OG0000869
1085    OG0000148    OG0000951
1153    OG0001108    OG0002724

[5006 rows x 2 columns]


Voy a contar las veces que aparece cada combinación única en cada archivo merge.csv 

Yo tengo un dataframe con combinaciones_unicas y ahora quiero añadir una columna para cada organismo (a partir del primer campo del archivo que acaba en merge.csv). Y ahora proceder a rellenar para cada pareja de ortogrupos cuántas veces aparece en su merge.csv. Por ejemplo tenemos   Orthogroup_x Orthogroup_y Mucci3 Aspnid1
0       OG0001460    OG0003027 2 0 esa pareja aparece 2 veces en Mucci3 (Mucci3_merge.csv) y cero veces en Aspnid1(Aspnid1_merge.csv). Pero hay que tener en cuenta que si el merge.csv esta pareja aparece así OG0001460    OG0003027  o así  OG0003027 OG0001460 sólo se cuente una vez.
quiero un archivo que contanga las parejas únicas de los ortogrupos y una columna por organismo procedente de contar cuántas veces aparece esa pareja de ortogrupos en cada organimso (en  cada merge_csv)

In [11]:
import pandas as pd
import glob

# Leer el DataFrame con las combinaciones únicas
combinaciones_unicas = pd.read_csv("combinaciones_unicas.csv")

# Obtener la lista de todos los archivos merge.csv en el directorio actual
archivos_merge = glob.glob("*_merge.csv")

# Crear un DataFrame para almacenar los recuentos por organismo
recuentos_por_organismo = pd.DataFrame(combinaciones_unicas[['Orthogroup_x', 'Orthogroup_y']])

# Inicializar las columnas de recuento para cada organismo con ceros
for archivo_merge in archivos_merge:
    organismo = archivo_merge.split("_")[0]  # Obtener el nombre del organismo
    recuentos_por_organismo[organismo] = 0

# Iterar sobre cada organismo y contar las combinaciones únicas en sus respectivos archivos merge.csv
for archivo_merge in archivos_merge:
    organismo = archivo_merge.split("_")[0]  # Obtener el nombre del organismo
    
    # Leer el archivo merge.csv
    df_merge = pd.read_csv(archivo_merge)
    
    # Contar las ocurrencias de cada combinación única
    recuentos = df_merge.groupby(['Orthogroup_x', 'Orthogroup_y']).size()
    
    # Actualizar los recuentos en el DataFrame recuentos_por_organismo
    for index, value in recuentos.items():
        recuentos_por_organismo.loc[(recuentos_por_organismo['Orthogroup_x'] == index[0]) & (recuentos_por_organismo['Orthogroup_y'] == index[1]), organismo] = value

# Guardar el DataFrame de recuentos por organismo en un archivo CSV
recuentos_por_organismo.to_csv("recuentos_por_organismo.csv", index=False)


In [ ]:
Voy a filtar para obtener las columnas que tengan al menos dos copias de una pareja de ortologos.


In [12]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv("recuentos_por_organismo.csv")

# Filtrar las filas que contienen al menos un dos en al menos una de las columnas de los organismos
df_filtrado = df.loc[(df.iloc[:, 3:] == 2).any(axis=1)]
df_filtrado
# Guardar el DataFrame filtrado en un nuevo archivo CSV
df_filtrado.to_csv("ortologos_filtrado.csv", index=False)


Parejas unicas

In [13]:
df_filtrado

,Orthogroup_x,Orthogroup_y,Aspnid1,Cunech1,Hesve2finisherSC,Liccor1,Mucci3,Mucend1,Mucfus1,Mucrac1,Phybl2,Rhimi59,Rhior3,Rhipu1,Rhisto1,Sakvas1,Spoumb1
198,OG0000010,OG0000010,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0
517,OG0000071,OG0000063,1,0,2,0,2,1,1,1,0,2,2,1,4,0,3
529,OG0000048,OG0000019,1,0,1,4,2,1,0,1,0,3,2,2,5,0,1
534,OG0001576,OG0000022,0,1,1,0,1,2,0,0,0,0,0,0,0,0,0
540,OG0000238,OG0000326,0,1,1,0,1,1,0,0,0,0,2,0,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4888,OG0001749,OG0002578,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4891,OG0003856,OG0003856,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4899,OG0000020,OG0001814,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4975,OG0001183,OG0000081,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
